# Key Distribution Kata

The **Quantum Key Distribution** kata is a series of exercises designed to teach you about a neat quantum technology where you can use qubits to exchange secure cryptographic keys. In particular, you will work through implementing and testing on a particuar quantum key distribution protocol called [BB84](https://en.wikipedia.org/wiki/BB84). 

### Background

What does a key distribution protocol look like in general? Normally there are two parties, commonly refered to as Alice and Bob, who want to share a random, secret string of bits called a _key_. This key can then be used for a variety of different [cryptographic protocols](https://en.wikipedia.org/wiki/Cryptographic_protocol) like encrpytion or authentication. Quantum versions of key exchange protocols look very similar, and utilize qubits as a way to securly transmit the bit string. 

TODO: finish desciption of bb84

There are already commercial devices that implement quantum key distribution protocols with single photons as the qubit systems. 

#### For more information:
* [Short animated video introducing BB84 protocol](https://www.youtube.com/watch?v=UVzRbU6y7Ks).



---
### Instructions

Each task is wrapped in one operation preceded by the description of the task.

Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl/⌘+Enter.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [1]:
%package Microsoft.Quantum.Katas::0.10.2002.2610

Adding package Microsoft.Quantum.Katas::0.10.2002.2610: done!

Microsoft.Quantum.Standard::0.10.2002.2610, Microsoft.Quantum.Katas::0.10.2002.2610

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Preparation

For the BB84 protocol, there is two main phases that it loops through until Alice and Bob have as much key as they want.
The first phase is the _quantum phase_ where Alice prepares indavidual qubits and then sends them to Bob to be measured.
The second phase is entirely classical post-processing and communcation that takes the perperation measuement results from the quantum phase and extracts a classical, random bitstring Alice and Bob can use.
Let's start by looking at how Alice will prepare her qubits for sending to Bob as a part of the quantum phase of the BB84 protocol.

Alice has two choices for each qubit, which basis to prepare it in, and what bit value she wants to encode.
This leads to four possible states each qubit can be in that Alice sends out.
The bases she has to choose from are selected such that if an evesdropper tries to measure a qubit in transit and chooses the wrong basis, then they just get a 0 or 1 with equal probability.

The first basis and states Alice uses to prepare her qubits are $|0\rangle$ and $|1\rangle$ where $|0\rangle$ represents the key bit value `0` and $|1\rangle$ represents the key bit value `1`.
The second basis (sometimes called the diagonal basis) uses the $|+\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$ to represent the classical bit value `0`, and $|-\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$ to represent the classical bit value `1`.


### Task 1.1. Diagonal Basis

Try your hand at converting qubits in the first basis to the diagonal basis.

**Input:** $N$ qubits (stored in an array of length $N$). Each qubit is either in $|0\rangle$ or in $|1\rangle$ state.

**Goal:**  Convert the qubits to the diagonal basis: 
* if `qs[i]` was in state $|0\rangle$, it should be transformed to $|+\rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$,
* if `qs[i]` was in state $|1\rangle$, it should be transformed to $|-\rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$.

In [2]:
%kata T11_DiagonalBasis_Test

operation DiagonalBasis (qs : Qubit[]) : Unit {    
    ApplyToEach( H, qs);
}

Success!

### Task 1.2. Equal superposition REVIEW OPTIONS

 
**Input**: A qubit in the $|0\rangle$ state.

**Goal**:  Change the qubit state to a superposition state that has equal probabilities of measuring 0 and 1. 

> Note that this is not the same as keeping the qubit in the $|0\rangle$ state with 50% probability and converting it to the $|1\rangle$ state with 50% probability!

In [10]:
%kata T12_EqualSuperposition_Test 

operation EqualSuperposition (q : Qubit) : Unit {
    H(q);
}

# wave function for qubits with ids (least to most significant): 0
∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
∣1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]


Success!

## Part II. BB84 Protocol

Now that you have seen some of the steps that Alice will use to prepare here qubits, it's time to add in the rest of the steps of the BB84 protocol.

### Task 2.1. Generate random array

You saw in part I that Alice has to make two random choices per qubit she prepares, one for which of two bases to use, and the other for what bit value she wants to send.
It turns out that Bob will also need one random bit value to decide what basis he will be measuring each qubit in.
To make this easier for later steps, you will need a way of generating random boolean values for both Alice and Bob to use.

**Input:** An integer $N$.

**Output** :  A `Bool` array of length N, where each element is chosen at random. 

> This will be used by both Alice and Bob to choose either the sequence of bits to send or the sequence of bases (`false` indicates $|0\rangle$ / $|1\rangle$ basis, and `true` indicates $|+\rangle$ / $|-\rangle$ basis) to use when encoding/measuring the bits.

In [ ]:
%kata T21_RandomArray_Test 

operation RandomArray (N : Int) : Bool[] {
    // ...
    return new Bool[N];
}

### Task 2.2. Prepare Alice's qubits

Now that you have a way of generating the random inputs needed for Alice and Bob, it's time for Alice to prepare her sequence of qubits.

**Inputs:** 

1. `qs`: an array of $N$ qubits in the $|0\rangle$ states,
2. `bases`: a `Bool` array of length $N$; 
    `bases[i]` indicates the basis to prepare the i-th qubit in:  
    * `false`: use $|0\rangle$ / $|1\rangle$ (computational) basis,
    * `true`: use $|+\rangle$ / $|-\rangle$ (Hadamard/diagonal) basis.
3. `bits`: a `Bool` array of length $N$;
    `bits[i]` indicates the bit to encode in the i-th qubit: `false` = 0, `true` = 1.

**Goal:**  Prepare the qubits in the described state.

In [ ]:
%kata T22_PrepareAlicesQubits_Test

operation PrepareAlicesQubits (qs : Qubit[], bases : Bool[], bits : Bool[]) : Unit {
    // ...
}

### Task 2.3. Measure Bob's qubits

**Inputs:**

1. `qs`: an array of $N$ qubits;  
   each qubit is in one of the following states: $|0\rangle$, $|1\rangle$, $|+\rangle$, $|-\rangle$. 
2. `bases`: a `Bool` array of length $N$; 
   `bases[i]` indicates the basis used to prepare the i-th qubit:
   * `false`: $|0\rangle$ / $|1\rangle$ (computational) basis,
   * `true`: $|+\rangle$ / $|-\rangle$ (Hadamard/diagonal) basis.

**Output:** Measure each qubit in the corresponding basis and return an array of results 
(encoding measurement result `Zero` as `false` and `One` as `true`). 
The state of the qubits at the end of the operation does not matter.

In [ ]:
%kata T23_MeasureBobsQubits_Test

operation MeasureBobsQubits (qs : Qubit[], bases : Bool[]) : Bool[] {
    // ...
    return new Bool[0];
}

### Task 2.4. Generate the shared key!
    
**Inputs:**

1. `basesAlice` and `basesBob`: `Bool` arrays of length $N$
   describing Alice's and Bobs's choice of bases, respectively;
2. `measurementsBob`: a `Bool` array of length $N$ describing Bob's measurement results.
    
**Output:** a `Bool` array representing the shared key generated by the protocol.

> Note that you don't need to know both Alice's and Bob's bits to figure out the shared key!

In [ ]:
%kata T24_GenerateSharedKey_Test

function GenerateSharedKey (basesAlice : Bool[], basesBob : Bool[], measurementsBob : Bool[]) : Bool[] {
    // ...
    return new Bool[0];
}

### Task 2.5. Was communication secure?

**Inputs:**

1. `keyAlice` and `keyBob`: `Bool` arrays of equal length $N$ describing 
   the versions of the shared key obtained by Alice and Bob, respectively.
2. `threshold`: an integer between 50 and 100 - the percentage of the key bits that have to match.
    
**Output:** `true` if the percentage of matching bits is greater than or equal to the threshold, and `false` otherwise.

In [ ]:
%kata T25_CheckKeysMatch_Test

function CheckKeysMatch (keyAlice : Bool[], keyBob : Bool[], threshold : Int) : Bool {
    // The following lines enforce the constraints on the input that you are given.
    // You don't need to modify them. Feel free to remove them, this won't cause your code to fail.
    Fact(Length(keyAlice) == Length(keyBob), "Input arrays should have the same length");

    // ...
    return false;
}

### Task 2.6. Putting it all together

**Goal:** Implement the entire BB84 protocol using tasks 2.1 - 2.5 
and following the comments in the operation template. 

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_BB84Protocol` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_BB84Protocol`).

In [ ]:
operation Run_BB84Protocol () : Unit {
    // 1. Alice chooses a random set of bits to encode in her qubits 
    //    and a random set of bases to prepare her qubits in.
    // ...

    // 2. Alice allocates qubits, encodes them using her choices and sends them to Bob.
    //    (Note that you can not reflect "sending the qubits to Bob" in Q#)
    // ...

    // 3. Bob chooses a random set of bases to measure Alice's qubits in.
    // ...

    // 4. Bob measures Alice's qubits in his chosen bases.
    // ...

    // 5. Alice and Bob compare their chosen bases and use the bits in the matching positions to create a shared key.
    // ...

    // 6. Alice and Bob check to make sure nobody eavesdropped by comparing a subset of their keys
    //    and verifying that more than a certain percentage of the bits match.
    // For this step, you can check the percentage of matching bits using the entire key 
    // (in practice only a subset of indices is chosen to minimize the number of discarded bits).
    // ...

    // If you've done everything correctly, the generated keys will always match, since there is no eavesdropping going on.
    // In the next section you will explore the effects introduced by eavesdropping.
}

In [ ]:
%simulate Run_BB84Protocol

## Part III. Eavesdropping

### Task 3.1. Eavesdrop!

In this task you will try to implement an eavesdropper, Eve. 

Eve will intercept a qubit from the quantum channel that Alice and Bob are using. 
She will measure it in either the $|0\rangle$ / $|1\rangle$ basis or the $|+\rangle$ / $|-\rangle$ basis,
reconstruct the qubit into the original state and send it back to the channel. 
Eve hopes that if she properly reconstructs the qubit after measurement she won't be caught!

**Inputs:**

1. `q`: a qubit in one of the following states: $|0\rangle$, $|1\rangle$, $|+\rangle$, $|-\rangle$.
2. `basis`: Eve's guess of the basis she should use for measuring.
   Recall that `false` indicates $|0\rangle$ / $|1\rangle$ basis and `true` indicates $|+\rangle$ / $|-\rangle$ basis. 

**Output:** the bit encoded in the qubit (`false` for $|0\rangle$ / $|+\rangle$ states, `true` for $|1\rangle$ / $|-\rangle$ states).

In this task you are guaranteed that the basis you're given matches the one
in which the qubit is encoded, that is, if you are given a qubit in state
$|0\rangle$ or $|1\rangle$, you will be given `basis = false`, and if you are given a qubit in state
$|+\rangle$ or $|-\rangle$, you will be given `basis = true`. This is different from a real
eavesdropping scenario, in which you have to guess the basis yourself.

In [ ]:
%kata T31_Eavesdrop_Test

operation Eavesdrop (q : Qubit, basis : Bool) : Bool {
    // ...
    return false;
}

### Task 3.2. Catch the eavesdropper

Add an eavesdropper into the BB84 protocol from task 2.6. 

Note that now we should be able to detect Eve and therefore we have to discard some of our keys!

> Similar to task 2.6, this is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_BB84ProtocolWithEavesdropper` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_BB84ProtocolWithEavesdropper`).

In [ ]:
operation Run_BB84ProtocolWithEavesdropper () : Unit {
    // ...
}

In [ ]:
%simulate Run_BB84ProtocolWithEavesdropper